In [ ]:
from zernike_algos import *
import numpy as np
import jax.numpy as jnp
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
basis = ZernikePolynomial(L=50, M=50, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)
radial = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0)
print(np.max(radial))
l = basis.modes[:,0]
m = basis.modes[:,1]
m = np.abs(m)
n = (l-m)//2

for i in range(len(n)):
    print(f"{l[i]}    {m[i]}    {n[i]}   {radial[2,i]}")

In [ ]:
print("zernike_radial, 0th derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 0).block_until_ready()
print("zernike_radial, 1st derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 1).block_until_ready()
print("zernike_radial, 2nd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 2).block_until_ready()
print("zernike_radial, 3rd derivative")
%timeit _ = zernike_radial(r[:,np.newaxis], basis.modes[:,0], basis.modes[:,1], 3).block_until_ready()

In [ ]:
import numpy as np

def jacobi_poly(x, n, alpha, beta):
    """
    Calculate the Jacobi polynomial P_n^(alpha, beta) at points x.

    Parameters:
    - x: array-like, Points where the Jacobi polynomial is evaluated.
    - n: array-like, Degree of the Jacobi polynomial.
    - alpha: array-like, Alpha parameter of the Jacobi polynomial.
    - beta: array-like, Beta parameter of the Jacobi polynomial.

    Returns:
    - P_n: array, Values of the Jacobi polynomial at points x.
    """
    result = []
    
    xj = 1 - 2 * x**2
    for i in range(len(alpha)):
        P_n1 = jacobi_poly_single(xj, 1, alpha[i], beta)
        P_n2 = jacobi_poly_single(xj, 0, alpha[i], beta)
        power = x**alpha[i]
        result.append(np.array((-1) ** 0 * power * P_n2))
        if n[i] >= 1:
            result.append(np.array((-1) ** 1 * power * P_n1))
        if n[i] >= 2:
            for N in range(2, n[i] + 1):
                P_n = jacobi_poly_single(xj, N, alpha[i], beta, P_n1, P_n2)
                result.append(np.array((-1) ** N * power * P_n))
                P_n2 = P_n1
                P_n1 = P_n
    return np.transpose(np.array(result))

def jacobi_poly_single(x, n, alpha, beta, P_n1=0, P_n2=0):
    if n == 0:
        return np.ones_like(x)
    elif n == 1:
        return (alpha+1) +  (alpha + beta + 2) * (x - 1) / 2
    else:
        c = 2*n + alpha + beta
        a1 = 2*n* (c-n) * (c-2)
        a2 = (c-1) * ( c*(c-2)*x + (alpha-beta)*(alpha+beta) )
        a3 = 2*(n+alpha-1)*(n+beta-1)*c
        
        P_n = (a2*P_n1 - a3*P_n2) / a1

        return P_n

basis = ZernikePolynomial(L=50, M=50, spectral_indexing="ansi", sym="cos")
r = np.linspace(0, 1, 100)
l = basis.modes[:,0]
m = basis.modes[:,1]
m = np.abs(m)
n = (l-m)//2

# for i in range(len(n)):
#     print(f"{l[i]}    {m[i]}    {n[i]}")

idx = np.lexsort((n,m))
id0 = np.arange(0,len(l))
id0 = id0[idx]

l = l[idx]
m = m[idx]
n = n[idx]

# for i in range(len(n)):
#     print(f"{l[i]}    {m[i]}    {n[i]}")
    
unique_values = np.unique(m)
opt_param = []
# For each unique value, find the maximum value in array2 where this value occurs in array1
for value in unique_values:
    indices = np.where(m == value)
    max_n = np.max(n[indices])
    id = np.argmax(n[indices])
    max_l = l[id]
    opt_param.append(np.array([value, max_n, max_l]))
opt_param = np.array(opt_param)
m_opt = opt_param[:,0]
n_opt = opt_param[:,1]
l_opt = opt_param[:,2]

# for i in range(len(n_opt)):
#     print(f"{m_opt[i]}    {n_opt[i]}")

# Broadcasting is used for element-wise operations
jacobi_values = jacobi_poly(r, n_opt, m_opt, 0)
jacobi_values = np.where((l - m) % 2 == 0, jacobi_values, 0)
jacobi_values = jacobi_values[:, np.argsort(id0)]
print(jacobi_values.shape)

l = l[np.argsort(id0)]
m = m[np.argsort(id0)]
n = n[np.argsort(id0)]
for i in range(len(n)):
    print(f"{l[i]}    {m[i]}    {n[i]}   {jacobi_values[0,i]}    {radial[0,i]}")
    
# print(jacobi_values)


In [ ]:
print(np.max(jacobi_values-radial))

In [ ]:
print(jacobi_values-radial)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mpmath

mpmath.mp.dps = 100
c = zernike_radial_coeffs(basis.modes[:, 0], basis.modes[:, 1], exact=True)

# current algorithm
zr0 = radial
# exact evaluation
zt0 = np.array([np.asarray(mpmath.polyval(list(ci), r), dtype=float) for ci in c]).T
# polynomial evaluation
zp0 = zernike_radial_poly(
    r[:, np.newaxis], basis.modes[:, 0], basis.modes[:, 1], dr=0, exact=False
)


cmap = plt.cm.jet  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]

# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    "Custom cmap", cmaplist, cmap.N
)

# define the bins and normalize
bounds = np.logspace(-16, 0, 17)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
fig, ax = plt.subplots(1, 3, squeeze=True, figsize=(10, 4))
im = ax[0].scatter(
    basis.modes[:, 0],
    basis.modes[:, 1],
    c=np.max(abs(zp0 - zt0), axis=0),
    norm=norm,
    cmap=cmap,
)
im = ax[1].scatter(
    basis.modes[:, 0],
    basis.modes[:, 1],
    c=np.max(abs(zr0 - zt0), axis=0),
    norm=norm,
    cmap=cmap,
)
im = ax[2].scatter(
    basis.modes[:, 0],
    basis.modes[:, 1],
    c=np.max(abs(jacobi_values - zt0), axis=0),
    norm=norm,
    cmap=cmap,
)
cbar = fig.colorbar(im, ticks=bounds)
cbar.ax.set_yticklabels(["{:.0e}".format(foo) for foo in bounds])
ax[0].grid(True)
ax[1].grid(True)
ax[2].grid(True)
ax[0].set_xticks(np.arange(0, 55, 5))
ax[0].set_yticks(np.arange(0, 55, 5))
ax[1].set_xticks(np.arange(0, 55, 5))
ax[1].set_yticks(np.arange(0, 55, 5))
ax[2].set_xticks(np.arange(0, 55, 5))
ax[2].set_yticks(np.arange(0, 55, 5))
ax[0].set_xlabel("$l$", fontsize=12)
ax[0].set_ylabel("$m$", fontsize=12)
ax[1].set_xlabel("$l$", fontsize=12)
ax[1].set_ylabel("$m$", fontsize=12)
ax[2].set_xlabel("$l$", fontsize=12)
ax[2].set_ylabel("$m$", fontsize=12)
ax[0].set_title(
    "$\max_{x \in (0,1)} |Z_{lm}(x) - \\tilde{Z}_{lm}^{poly}(x)|$", fontsize=14
)
ax[1].set_title(
    "$\max_{x \in (0,1)} |Z_{lm}(x) - \\tilde{Z}_{lm}^{jacobi}(x)|$", fontsize=14
);
ax[2].set_title(
    "$\max_{x \in (0,1)} |Z_{lm}(x) - \\tilde{Z}_{lm}^{optimized}(x)|$", fontsize=14
);